In [1]:
import sys
sys.path.append('../src')

from pipelines import preprocessing_pipeline
from pipelines import classifier_pipeline


import os
import matplotlib.pyplot as plt
import numpy as np

import cv2
from skimage.feature import hog

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

root_path = '../'
ck_data_path = os.path.join(root_path, 'data/CK+48')
jaffe_data_path = os.path.join(root_path, 'data/JAFFE')

# Gabor filters with CK+48

### Data

In [2]:
images, labels = preprocessing_pipeline(ck_data_path, 'ck', verbose=True)

  0%|          | 0/981 [00:00<?, ?it/s]

100%|██████████| 981/981 [00:04<00:00, 227.39it/s]

Dataset: ck
Dataset length: 981
Image shape: (48, 48)


In [3]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Function to extract Gabor features from an image
def gabor_features(image, ksize=9, sigma=2.0, theta=0, lambd=5.0, gamma=0.5):
    gabor_filter = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, 0, ktype=cv2.CV_64F)
    filtered_image = cv2.filter2D(image, cv2.CV_8UC3, gabor_filter)
    return filtered_image.flatten()


In [4]:
# Extract Gabor features from each image
X = []
for image in images:
    features = gabor_features(image)
    X.append(features)

X = np.array(X)
y = np.array(labels)

print(X.shape)
print(y.shape)

(981, 2304)
(981,)


In [5]:
train_results, test_results = classifier_pipeline(X, y, verbose=True)

Train Result:
Accuracy Score: 100.00%
Test Result:
Accuracy Score: 99.66%


### JAFFE

In [6]:
images, labels = preprocessing_pipeline(jaffe_data_path, 'jaffe', verbose=True)

100%|██████████| 213/213 [00:06<00:00, 31.04it/s]

Dataset: jaffe
Dataset length: 213
Image shape: (256, 256)


In [7]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Function to extract Gabor features from an image
def gabor_features(image, ksize=9, sigma=2.0, theta=0, lambd=5.0, gamma=0.5):
    gabor_filter = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, 0, ktype=cv2.CV_64F)
    filtered_image = cv2.filter2D(image, cv2.CV_8UC3, gabor_filter)
    return filtered_image.flatten()

In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Assuming 'X' is your input data
X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Define the custom function for scaling by 255
def scale_by_255(X):
    return X / 255.0

# Create a pipeline with FunctionTransformer and your classifier (e.g., SVC)
pipeline = Pipeline([
    ('scaler', FunctionTransformer(func=scale_by_255)),
    ('classifier', SVC())
])

# Fit and evaluate the pipeline
pipeline.fit(X_train, y_train)
accuracy = pipeline.score(X_test, y_test)

print(f"Accuracy: {accuracy}")


In [8]:
# Extract Gabor features from each image
X = []
for image in images:
    features = gabor_features(image)
    X.append(features)

X = np.array(X)
y = np.array(labels)

print(X.shape)
print(y.shape)

(213, 65536)
(213,)


In [9]:
train_results, test_results = classifier_pipeline(X, y, verbose=True)

Train Result:
Accuracy Score: 100.00%
Test Result:
Accuracy Score: 65.62%


In [10]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an SVM classifier
clf = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=1.0))

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 95.94%
